In [ ]:
!pwd

/media/mohsin/5aca7fc2-9fd6-40dd-b214-8e2adf4d6a1b/Projects/Fashion-Product-Classification/research


In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
!pwd

/media/mohsin/5aca7fc2-9fd6-40dd-b214-8e2adf4d6a1b/Projects/Fashion-Product-Classification


In [8]:
import pandas as pd

df = pd.read_csv("artifacts/data_ingestion/styles.csv", on_bad_lines = "skip",encoding='utf-8')
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [14]:
import os

# Drop missing articleTypes
df = df.dropna(subset=["articleType"])

# Ensure image files exist
image_dir = "artifacts/data_ingestion/images"
df["image_path"] = df["id"].astype(str) + ".jpg"
df = df[df["image_path"].apply(lambda x: os.path.isfile(os.path.join(image_dir, x)))]


In [15]:
df[['id', 'articleType', 'image_path']]

,id,articleType,image_path
0,15970,Shirts,15970.jpg
1,39386,Jeans,39386.jpg
2,59263,Watches,59263.jpg
3,21379,Track Pants,21379.jpg
4,53759,Tshirts,53759.jpg
...,...,...,...
44419,17036,Casual Shoes,17036.jpg
44420,6461,Flip Flops,6461.jpg
44421,18842,Tshirts,18842.jpg
44422,46694,Perfume and Body Mist,46694.jpg


In [18]:
top_classes = df["articleType"].value_counts().head(10)
print(top_classes)

articleType
Tshirts         7066
Shirts          3215
Casual Shoes    2845
Watches         2542
Sports Shoes    2036
Kurtas          1844
Tops            1762
Handbags        1759
Heels           1323
Sunglasses      1073
Name: count, dtype: int64


In [19]:
top_class_names = top_classes.index.tolist()
df_top10 = df[df["articleType"].isin(top_class_names)].copy()

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_top10["label"] = le.fit_transform(df_top10["articleType"])

In [21]:
import shutil
import os

base_dir = "data/top10_classes"
os.makedirs(base_dir, exist_ok=True)

for _, row in df_top10.iterrows():
    class_name = row["articleType"]
    src = os.path.join(image_dir, row["image_path"])
    dst_dir = os.path.join(base_dir, class_name)
    os.makedirs(dst_dir, exist_ok=True)
    shutil.copy(src, os.path.join(dst_dir, row["image_path"]))


In [22]:
import tensorflow as tf

AUTOTUNE = tf.data.AUTOTUNE

# Paths
image_paths = df_top10["image_path"].apply(lambda x: os.path.join(image_dir, x)).tolist()
labels = df_top10["label"].tolist()

# Create dataset
path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
label_ds = tf.data.Dataset.from_tensor_slices(labels)

# Combine
image_label_ds = tf.data.Dataset.zip((path_ds, label_ds))

# Preprocessing function
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0  # Normalize
    return img, tf.one_hot(label, depth=10)

ds = image_label_ds.map(process_path, num_parallel_calls=AUTOTUNE)


2025-05-27 12:57:18.969426: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 12:57:23.115251: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 12:57:23.153193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-27 12:57:35.014707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [23]:
ds_size = len(df_top10)
train_size = int(0.7 * ds_size)
val_size = int(0.2 * ds_size)

train_ds = ds.take(train_size)
val_ds = ds.skip(train_size).take(val_size)
test_ds = ds.skip(train_size + val_size)

# Batch & prefetch
train_ds = train_ds.shuffle(1000).batch(32).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(buffer_size=AUTOTUNE)


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

2025-05-28 11:55:13.301701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-28 11:55:13.301805: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [26]:
# Load base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze base model

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

16705208/16705208 [==============================] - 129s 8us/step


In [27]:
# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2


2025-05-27 13:07:13.033878: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 867 of 1000
2025-05-27 13:07:15.985338: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


KeyboardInterrupt: 